<a href="https://colab.research.google.com/github/KWinston3002/Brazilian-E-Commerce-Dataset-Analysis/blob/main/KeyCommerceQuestionsAnalysis_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Database
**Run the following cells to create a sqllite database**

In [ ]:
#Do not edit or change this cell
import pandas as pd
import sqlite3
import duckdb
import matplotlib
import requests
from IPython.core.magic import (register_cell_magic, Magics, magics_class, cell_magic)

!wget -nc --no-verbose "https://github.com/arezzy17/AdvancedAnalytics_BUS325/raw/main/brazilecommerce.duckdb"
conn = duckdb.connect("brazilecommerce.duckdb")

@magics_class
class DuckDBMagics(Magics):
    def __init__(self, shell):
        super(DuckDBMagics, self).__init__(shell)
        self.conn = conn

    @cell_magic
    def duckdb(self, line, cell):
        # Execute the cell content as DuckDB query
        result = self.conn.execute(cell).fetchdf()
        return result

# Register the magic with the current IPython shell
ip = get_ipython()
ip.register_magics(DuckDBMagics)

2024-10-11 02:43:55 URL:https://raw.githubusercontent.com/arezzy17/AdvancedAnalytics_BUS325/main/brazilecommerce.duckdb [30945280/30945280] -> "brazilecommerce.duckdb" [1]


What is your key question? (Copy from [spreadsheet column E](https://docs.google.com/spreadsheets/d/158uhza-gQKNb5j7RpQlj5gV8NUNO5WoAGKkMf8Iin8o/edit?usp=sharing) )

In [ ]:
Does our estimated delivery time vary by state?

What is the **first testable question** that you would want to answer to address your key question?

In [ ]:
What are the average deliverey time from each state?

**Write a SQL query** to answer your first testable question.

In [ ]:
%%duckdb

SELECT
  c.customer_state,
  AVG(julian(order_delivered_customer_date) - julian(order_purchase_timestamp)) AS avg_days_took_to_deliver,

FROM Orders AS o

JOIN Customers AS c ON o.customer_id = c.customer_id

GROUP BY c.customer_state
ORDER BY avg_days_took_to_deliver DESC;










,customer_state,avg_days_took_to_deliver
0,RR,29.387546
1,AP,27.185068
2,AM,26.425991
3,AL,24.543855
4,PA,23.772917
5,MA,21.572976
6,SE,21.519788
7,CE,21.266579
8,AC,21.035713
9,PB,20.426768


In your words, **what was the answer to your first testable question?** How does this help answer your key question?

In [ ]:
The state with the longest delivery time is RR (Roraima) with 29.39 days. The state with the shortest delivery time is SP (São Paulo) with 8.76 days. This directly helps answer the key question by demonstrating that delivery times do indeed vary by state. The range of delivery times shows that different states have different logistical challenges or factors affecting how long it takes for a product to be delivered.

(With knowledge from your first testable question) What is the **second testable question** that you would want to answer to address your key question?

In [ ]:
Where are most of the sellers located?

**Write a SQL query** to answer your second testable question.

In [ ]:
%%duckdb

SELECT
  seller_state,
  COUNT(*) AS seller_count

FROM Sellers

GROUP BY seller_state
ORDER BY seller_count DESC;

,seller_state,seller_count
0,SP,1849
1,PR,349
2,MG,244
3,SC,190
4,RJ,171
5,RS,129
6,GO,40
7,DF,30
8,ES,23
9,BA,19


In your words, **what was the answer to your second question?** How does this help answer your key question?

In [ ]:
Based on the data,there is a significant variation in the number of sellers by state. São Paulo (SP) dominates with 1,849 sellers, while several states such as Acre (AC), Piauí (PI), and Amazonas (AM) have only 1 seller each. We could hypothesize that states with a higher number of sellers have shorter delivery times due to more localized fulfillment, while states with fewer sellers may experience delays due to cross-regional shipping

(With knowledge from your first two testable questions) What is the **third testable question** that you would want to answer to address your key question?

In [ ]:
Do heavier products take longer than lighter products to deliver?

**Write a SQL query** to answer your third testable question.

In [ ]:
%%duckdb

SELECT

  p1.product_weight_g,
  AVG(julian(order_delivered_customer_date) - julian(order_purchase_timestamp)) AS avg_days_took_to_deliver,
  p.product_category_name

FROM Products AS p

LEFT JOIN Order_Items AS oi ON p.product_id = oi.product_id
LEFT JOIN Orders AS o ON oi.order_id = o.order_id
LEFT JOIN Products AS p1 ON oi.product_id = p1.product_id

GROUP BY p.product_category_name, p1.product_weight_g
ORDER BY avg_days_took_to_deliver DESC
LIMIT 20






,product_weight_g,avg_days_took_to_deliver,product_category_name
0,383.0,168.082801,relogios_presentes
1,9250.0,107.451209,eletrodomesticos_2
2,370.0,90.960995,telefonia
3,9300.0,72.614155,moveis_decoracao
4,3462.0,64.866076,informatica_acessorios
5,7450.0,63.935486,cama_mesa_banho
6,6400.0,62.991157,beleza_saude
7,4825.0,59.371285,esporte_lazer
8,16300.0,58.527928,moveis_decoracao
9,130.0,57.931852,informatica_acessorios


In your words, **what was the answer to your third question?** How does this help answer your key question?

In [ ]:
There isn’t a clear linear correlation between product weight and delivery time. While heavier products tend to be delivered faster in this dataset, it might not hold across all products.This analysis provides insight into the relationship between product weight and delivery time, which can help inform inventory management, shipping strategies, and customer expectations. If heavier items are consistently taking less time to deliver, it could suggest that logistical efficiencies exist for those products, potentially due to their handling or shipping processes

How would you summarize your current answer to your key question based on information gathered by answering your first 5 testable questions?

In [ ]:
The data supports the conclusion that delivery times do vary by state, influenced by the number of sellers, geographic factors, and potentially the nature of the products being shipped. Understanding these dynamics can help optimize logistics and improve overall delivery efficiency.
This understanding allows for more informed decision-making regarding shipping strategies, inventory placement, and customer communication regarding delivery expectations.